In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# fad_score

> Produce FAD score based on files of embeddings of real and fake data

$$ FAD = || \mu_r - \mu_f ||^2 + tr\left(\Sigma_r + \Sigma_f - 2 \sqrt{\Sigma_r \Sigma_f}\right)$$

The embeddings are small enough that this can typically be run on a single processor, on a CPU. However, all the supporting code is GPU-friendly if so desired. 

In [ ]:
#| default_exp fad_score

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch 
import argparse
from fad_pytorch.sqrtm import sqrtm
from aeiou.core import fast_scandir

In [ ]:
#| export
def read_embeddings(emb_path='real_emb_clap/', debug=False):
    "reads any .pt files in emb_path and concatenates them into one tensor"
    if debug: print("searching in ",emb_path) 
    _, file_list = fast_scandir(emb_path, ['pt'])
    if  file_list == []:
        _, file_list = fast_scandir('/fsx/shawley/code/fad_pytorch/'+emb_path, ['pt']) # yea, cheap hack just for my testing in nbs/ dir
    assert file_list != []
    embeddings = []
    for file_path in file_list:
        emb_batch = torch.load(file_path, map_location='cpu') 
        embeddings.append(emb_batch)
    return torch.cat(embeddings, dim=0)

In [ ]:
#| eval: false
# lil test of that
e = read_embeddings()
e.shape

torch.Size([256, 512])

In [ ]:
#| export 
def calc_mu_sigma(emb): 
    "calculates mean and covariance matrix of batched embeddings"
    mu = torch.mean(emb, axis=0)
    sigma = torch.cov(emb.T)
    return mu, sigma

In [ ]:
#| eval: false
# quick test:
x = torch.rand(32,512) 
mu, sigma = calc_mu_sigma(x) 
mu.shape, sigma.shape 

(torch.Size([512]), torch.Size([512, 512]))

In [ ]:
#| export
def calc_score(real_emb_path, # where real embeddings are stored
               fake_emb_path, # where fake embeddings are stored
               method='maji', # sqrtm calc method: 'maji'|'li'
               debug=False
               ): 
    print(f"Calculating FAD score for files in {real_emb_path}/ vs. {fake_emb_path}/")
    emb_real = read_embeddings(emb_path=real_emb_path, debug=debug)
    emb_fake = read_embeddings(emb_path=fake_emb_path, debug=debug)
    if debug: print(emb_real.shape, emb_fake.shape)
    
    mu_real, sigma_real = calc_mu_sigma(emb_real) 
    mu_fake, sigma_fake = calc_mu_sigma(emb_fake) 
    if debug:
        print("mu_real.shape, sigma_real.shape =",mu_real.shape, sigma_real.shape)
        print("mu_fake.shape, sigma_fake.shape =",mu_fake.shape, sigma_fake.shape)
    
    mu_diff = mu_real - mu_fake
    if debug:
        print("mu_diff = ",mu_diff) 
        score1 = mu_diff.dot(mu_diff)
        print("score1: mu_diff.dot(mu_diff) = ",score1)
        score2 = torch.trace(sigma_real)
        print("score2: torch.trace(sigma_real) = ", score2)
        score3 = torch.trace(sigma_fake)
        print("score3: torch.trace(sigma_fake) = ",score3)
        score_p = sqrtm( torch.matmul( sigma_real, sigma_fake) )
        print("score_p.shape (matmul) = ",score_p.shape) 
        score4 = -2* torch.trace ( torch.real ( sqrtm( torch.matmul( sigma_real, sigma_fake) , method=method ) ) )
        print("score4 (-2*tr(sqrtm(matmul(sigma_r sigma_f))))  = ",score4) 
        score = score1 + score2 + score3 + score4
    score = mu_diff.dot(mu_diff) + torch.trace(sigma_real) + torch.trace(sigma_fake) -2* torch.trace ( torch.real ( sqrtm( torch.matmul( sigma_real, sigma_fake), method=method  ) ) )
    return score

Test the score function:

In [ ]:
#| eval: false
score = calc_score( 'real_emb_clap/', 'fake_emb_clap/', method='maji')
print(score)

Calculating FAD score for files in real_emb_clap// vs. fake_emb_clap//
tensor(0.0951)


Try sending using the exact same data for both distributions: Do we get zero? 

In [ ]:
score = calc_score( 'real_emb_clap/', 'real_emb_clap/', method='maji', debug=True)
print(score)

Calculating FAD score for files in real_emb_clap// vs. real_emb_clap//
searching in  real_emb_clap/
searching in  real_emb_clap/
torch.Size([256, 512]) torch.Size([256, 512])
mu_real.shape, sigma_real.shape = torch.Size([512]) torch.Size([512, 512])
mu_fake.shape, sigma_fake.shape = torch.Size([512]) torch.Size([512, 512])
mu_diff =  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

Ok, so not zero, but small.

In [ ]:
#| export
def main(): 
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('real_emb_path', help='Path of files of embeddings of real data', default='real_emb_clap/')
    parser.add_argument('fake_emb_path', help='Path of files of embeddings of fake data', default='fake_emb_clap/')
    parser.add_argument('-d','--debug', action='store_true', help='Enable debugging')
    parser.add_argument('-m','--method', default='maji', help='Method for sqrtm calculation: "maji" or "li" ')

    args = parser.parse_args()
    score = calc_score( args.real_emb_path, args.fake_emb_path, method=args.method, debug=args.debug )
    print("FAD score = ",score.cpu().numpy())

[autoreload of fad_pytorch.sqrtm failed: Traceback (most recent call last):
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/fsx/shawley/envs_sm/aa/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj)

In [ ]:
#| export
if __name__ == '__main__' and "get_ipython" not in dir():
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()